In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
data_dict = {'Title': [],'Category': [], 'Price': [], 'Image': []}
driver = webdriver.Chrome(ChromeDriverManager().install())

total_pages = 2
page = 1

while page <= int(total_pages):
    time.sleep(2)
    
    print(f'Scrapping page: {page}')

    scrapping_site_url = f'https://www.daraz.pk/jk-store/?from=wangpu&lang=en&langFlag=en&page={page}&pageTypeId=2&q=All-Products'

    driver.get(scrapping_site_url)
    
    html = driver.page_source
    product_url = []
    parsed_html = BeautifulSoup(html, "html.parser")
    get_specific_url = parsed_html.find_all('a',age="0")
    for i in get_specific_url:
        product_url.append('http:'+i['href'])
    #Removing duplicates
    product_url= list(set(product_url))
    
    try:
        total_pages = driver.find_element_by_xpath('//ul[@class="ant-pagination "]/li[position() = (last() - 1)]').text
        print(total_pages)
    except:
        continue
    for j in range(len(product_url)):
        driver.get(product_url[j])
        
        name = driver.find_element_by_xpath('//*[@id="module_product_title_1"]/div/div/span').text
        data_dict['Title'].append(name)
    
        prices = driver.find_element_by_xpath('//*[@id="module_product_price_1"]/div/div/span').text
        data_dict['Price'].append(prices)
    
        image = driver.find_element_by_xpath('//*[@id="module_item_gallery_1"]/div/div[1]/div/img')
        data_dict['Image'].append(image.get_attribute("src"))
    
        category = driver.find_element_by_xpath('//*[@id="J_breadcrumb"]').text
        data_dict['Category'].append(category)
                
    page += 1


In [56]:
df = pd.DataFrame.from_dict(data_dict, orient="columns")
#df['Price'] = df['Price'].apply(lambda x: x.replace('Rs.', '').strip())
df.to_excel("scrapped_data_all_pages_9.xlsx", index=False)